In [1]:
import pandas as pd
import numpy as np
import json
import os
from collections import defaultdict, deque
import pickle
from IPython import get_ipython
from tqdm.auto import tqdm
import gc
import subprocess
from IPython.display import clear_output
import matplotlib.pyplot as plt
import wandb

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is", device)

device is cuda


# Train

In [11]:
%run train.py --lr 0.0025 --layers 1024 --num_timesteps 1000 --is_y_cond --save_as 'dqn_test' 

Index(['Month_sin', 'Month_cos', 'Hour_sin', 'Hour_cos', 'Day_sin', 'Day_cos',
       'Coord X', 'Coord Y', 'Duration', 'Incident', 'Month', 'Day', 'Hour'],
      dtype='object')
Index(['Month_sin', 'Month_cos', 'Hour_sin', 'Hour_cos', 'Day_sin', 'Day_cos',
       'Coord X', 'Coord Y', 'Duration', 'Incident'],
      dtype='object')
K [0]
features 9
{'d_in': np.int64(9), 'is_y_cond': True, 'num_classes': 56, 'rtdl_params': {'d_layers': [1024, 1024], 'dropout': 0.0}, 'dim_t': 128}
mlp
label embedding Embedding(56, 128)
diffusion ready
Model trained


# Sample

In [ ]:
for i in range(10): # génère i datasets
    print("iteration", i)
    filename = f"df_fake_{i}.pkl"
    
    process = subprocess.Popen(
        ["python", "sample.py", 
         "--load_as", 'dqn_test', 
         "--save_sample_as", filename, 
         "--os_factor", "3", 
         "--to_keep", "40", 
         "--value_span", "100"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1  # pour forcer le flush ligne par ligne
    )

    # process.wait() # évite le lazy

    line_count = 0
    for line in process.stdout:
        line_count += 1
        if line_count % 50 == 0:
            clear_output(wait=True)
    
        print(line.strip())

    print(f"--- {filename} done---\n")

# Generate environment

In [ ]:
# lister les fichiers samplés à concaténer

In [ ]:
%%time

%run generate_environment.py --sample_list df_fake_0.pkl df_fake_1.pkl df_fake_2.pkl df_fake_3.pkl df_fake_4.pkl df_fake_5.pkl df_fake_6.pkl df_fake_7.pkl df_fake_8.pkl df_fake_9.pkl --save_as "df_pc_fake_10y.pkl"

# Metrics

In [4]:
dic_tarif_sent_disp = {
                'v_sent': 0,
                'v_sent_full': 0,
                'v_degraded':0,
                'cancelled':0,
                'function_not_found':0,
                'v1_not_sent_from_1st_station':0,
                'v_not_found_in_last_station':0,
                'z1_EPA_sent': 0,
                'z1_FPT_sent': 0,
                'z1_VSAV_sent': 0,
                'VSAV_needed':0,
                'FPT_needed':0,
                'EPA_needed':0,
                'VSAV_disp':10,
                'FPT_disp':10,
                'EPA_disp':10,
                'skill_lvl':0
                }

os.chdir('./Reward_weights')

with open(f"rw_sent_disp.json", "w") as f:
        json.dump(dic_tarif_sent_disp, f)

os.chdir("../")

# Simulation

In [ ]:
cmd = [
    "python3", "-u", "simulation_start.py",
    "--reward_weights", f"rw_sent_disp.json",
    "--dataset", "df_pc_real.pkl",
    "--start", "1",
    "--end", "53088",
    "--constraint_factor_veh", "3",
    "--constraint_factor_ff", "1",
    "--is_best",
    "--save_metrics_as", f"sim_sent_disp_bst"
]

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
stdout_lines = deque(maxlen=5000)
for line in process.stdout:
    stdout_lines.append(line)
    line_count += 1
    if line_count % 100 == 0:
        clear_output(wait=True)

    print(line.strip())

# Agent params

In [ ]:
action_size = 80
train_seed = 41

os.chdir("./Data")


hyper_params = {"state_size" : (action_size + 2) *40,
                "action_size" : action_size,
                "layer_type" : "ff", # noisy else ff
                "layer_size" : 1024,
                "num_layers" : 8,
                "use_batchnorm" : True,
                "n_steps" : 20, #5, 1, 50, 64
                "batch_size" : 256, #512, 256, 64, 32, 16, 8
                "buffer_size" : 100000, #100000, 128, 5000, 10000, 65 buffer_size > batch_size
                "update_every" : 32, #1, 200, 500, 64, 32 Q updates
                "per" : 1, # 0 for curiosity > 0, else 1, 2
                "rdm" : 0, # only if not per
                "munchausen" : 1, #1
                "curiosity" : 0, #Adds intrinsic curiosity to the extrinsic reward. 0 - only reward/ no curiosity, 
                                                                                    #1 - reward and curiosity, 
                                                                                    #2 - only curiosity
                "curiosity_size" : 1024,
                "lr" : 1e-4, #1e-3, 5e-4, 5e-3, 1e-4
                "lr_dec" : 1, #0, 1, 2, 3
                "entropy_tau" : 0.03, #0.03, 0.05 idem #-  Munch param
                "entropy_tau_coeff" : 1e-2, #1e-2 #-  Munch param
                "lo" : -1, #-  Munch param
                "alpha" : 0.9, #-  Munch param
                "gamma" : 0.99,
                "tau" : 0.005, #1e-2, 5e-3
                "N" : 32,# Number of quantiles 32, 64
                "entropy_coeff" : 0.001,
                "decay_update" : 100,
                "device" : str(device),
                "seed" : train_seed}


json.dump(hyper_params, open("hyper_params.json", "w"))

os.chdir("../")

# Agent model

In [ ]:
model = "fqf"
years = "10y"

cmd = [
    "python3", "-u", "agent_run.py",
    "--model_name", f"agent_{model}_{years}",
    "--agent_model", model,
    "--hyper_params", "hyper_params.json",
    "--reward_weights", f"rw_sent_disp.json",
    "--dataset", f"df_pc_fake_{years}.pkl",
    "--start", "1",
    "--end", "530880",
    "--eps_start", "1",
    "--constraint_factor_veh", "3",
    "--constraint_factor_ff", "1",
    "--save_metrics_as", f"agent_metrics_{model}_{years}",
    "--train",

]
# "--load"

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
stdout_lines = deque(maxlen=5000)
for line in process.stdout:
    stdout_lines.append(line)
    line_count += 1
    if line_count % 100 == 0:
        clear_output(wait=True)

    print(line.strip())

# Stats

In [ ]:
folder_path = "./Plots"

data = {}

for file_name in os.listdir(folder_path):
    if file_name.endswith(".pkl"):
        full_path = os.path.join(folder_path, file_name)
        with open(full_path, "rb") as f:
            d = pickle.load(f)
            name = os.path.splitext(file_name)[0]
            data[name] = d

df = pd.DataFrame.from_dict(data, orient='index')

# Plots

In [ ]:
%%time

%run plot_evo.py \
agent_metrics_v_degraded_r100_cf3_reward_real.npy \
sim_metrics_r100_cf3_reward_real.npy \
--interpolation 1000